# Team Captain - M5 Accuracy: Sales Prediction

In [0]:
import catboost

import config
import prepare_data
import utility

In [0]:
regressor = catboost.CatBoostRegressor()
regressor.load_model(str(config.model_path / 'catboost_regressor.cbm'))

In [0]:
calendar = utility.read_calendar(config.m5_path / 'calendar.csv')

In [0]:
sales_data = utility.read_sales_data(config.m5_path / 'sales_train_validation.csv')

In [0]:
def predict(start_day):
    days = range(start_day, start_day + 28)

    data = prepare_data.generate_prediction_data(sales_data, calendar, days)

    predictions = regressor.predict(data)
    predictions = predictions.reshape(28, -1).T

    predictions[predictions < 0] = 0

    return predictions

In [0]:
# calendar is 0-based
predictions_evaluation = predict(1941)
predictions_validation = predict(1913)

In [0]:
utility.save_predictions(
    predictions_evaluation,
    predictions_validation,
    sales_data.id
)